In [93]:
import numpy as np
import tensorly as tl
from tensorly.cp_tensor import cp_to_tensor
import matplotlib.pyplot as plt
import random
random.seed(42)

In [94]:
alpha = 1000  # Influence parameter
lambda_ = 100  # Drift parameter
sigma = 100  # Diffusion parameter
T = 1.0  # Total time
dt = 0.001  # Time step
n_iterations = int(T / dt)

n_particles = 1000 # initial random tensors
I, J, K = 10, 12, 14
rank = 5  # Rank of the decomposition

# Create a random tensor
tensor = tl.tensor(np.random.random((I, J, K)))

In [95]:
# Step 2: Initialize particles 1000 random A, 1000 random B, 1000 random C

particles = []
for _ in range(n_particles):
    A = np.random.randn(I, rank) # 100 10x5 matrices
    B = np.random.randn(J, rank) # 100 12x5 matrices
    C = np.random.randn(K, rank) # 100 14x5 matrices
    particles.append({'A': A, 'B': B, 'C': C})

In [96]:
# particles[i], where i = [A_i, B_i, C_i] and A_i.shape = 10x5, B_i.shape = 12x5, C_i.shape = 14x5
# a{r}_{i} = particles[i-1]['A'][:,r-1]
# where r = rank from 1 to 5, i = particles from 1 to n_particles

"""
# example indexing:
a1_1 = particles[0]['A'][:,0]
b1_1 = particles[0]['B'][:,0]
c1_1 = particles[0]['C'][:,0]

a2_1 = particles[0]['A'][:,1]
b2_1 = particles[0]['B'][:,1]
c2_1 = particles[0]['C'][:,1]

a3_1 = particles[0]['A'][:,2]
b3_1 = particles[0]['B'][:,2]
c3_1 = particles[0]['C'][:,2]

a4_1 = particles[0]['A'][:,3]
b4_1 = particles[0]['B'][:,3]
c4_1 = particles[0]['C'][:,3]

a5_1 = particles[0]['A'][:,4]
b5_1 = particles[0]['B'][:,4]
c5_1 = particles[0]['C'][:,4]

# ----------------------------

a1_2 = particles[1]['A'][:,0]
b1_2 = particles[1]['B'][:,0]

# and so on"""

"\n# example indexing:\na1_1 = particles[0]['A'][:,0]\nb1_1 = particles[0]['B'][:,0]\nc1_1 = particles[0]['C'][:,0]\n\na2_1 = particles[0]['A'][:,1]\nb2_1 = particles[0]['B'][:,1]\nc2_1 = particles[0]['C'][:,1]\n\na3_1 = particles[0]['A'][:,2]\nb3_1 = particles[0]['B'][:,2]\nc3_1 = particles[0]['C'][:,2]\n\na4_1 = particles[0]['A'][:,3]\nb4_1 = particles[0]['B'][:,3]\nc4_1 = particles[0]['C'][:,3]\n\na5_1 = particles[0]['A'][:,4]\nb5_1 = particles[0]['B'][:,4]\nc5_1 = particles[0]['C'][:,4]\n\n# ----------------------------\n\na1_2 = particles[1]['A'][:,0]\nb1_2 = particles[1]['B'][:,0]\n\n# and so on"

In [97]:
particles[0]

{'A': array([[ 0.33536131, -0.64027975, -0.18174423, -2.25035451,  0.47668598],
        [-0.05126425,  0.00558905,  0.17011568,  0.11151251, -0.14152062],
        [ 1.80658785, -0.15542475, -0.48396326,  0.64863995, -0.99937218],
        [ 0.34086609, -0.78023415,  2.14094675, -0.68974288, -0.8184761 ],
        [ 0.11231857,  0.32705183,  0.14677837, -0.98642057, -0.69590817],
        [ 1.47348963,  0.00818342,  0.4790036 ,  0.98605059, -0.10170411],
        [-1.10413498, -0.97982928, -0.70013497,  0.17535954,  0.69033585],
        [-2.13328136,  0.99853405, -1.42756101,  0.31413716, -0.41212965],
        [ 0.79989073,  0.53637533, -1.39071669,  1.34089937,  1.16187939],
        [-0.20813981, -1.42928891, -0.06805849,  1.51927366,  2.26095943]]),
 'B': array([[-0.77914331,  0.92080399, -0.5725921 ,  0.07418772,  0.06370136],
        [-1.23836638, -0.23297674, -0.43620561,  0.72039227, -0.05083196],
        [ 0.15849969,  0.56974778, -2.36262493,  0.27614901, -0.67312505],
        [-2.1

In [98]:
def objective_function(particle):
    # Reconstruct tensor from the given particle
    A = particle['A']
    B = particle['B']
    C = particle['C']
    reconstructed_tensor = cp_to_tensor((np.ones(rank), [A, B, C]))

    # Compute the Frobenius norm of the difference
    error = tl.norm(tensor - reconstructed_tensor)

    return error

In [99]:
objective_function(particles[0])

124.05707442194252

In [100]:
def compute_consensus_point(particles, alpha):
    # Find the particle with the minimum energy (V* in the image)
    min_energy_particle = min(particles, key=objective_function)
    min_energy = objective_function(min_energy_particle)

    # Initialize the numerators and the denominator
    numerator_A = np.zeros_like(particles[0]['A'])
    numerator_B = np.zeros_like(particles[0]['B'])
    numerator_C = np.zeros_like(particles[0]['C'])
    denominator = 0.0

    for particle in particles:
        A, B, C = particle['A'], particle['B'], particle['C']

        # Compute the energy for the current particle
        energy = objective_function(particle)

        # Apply the numerical stabilization trick
        weight = np.exp(-alpha * (energy - min_energy))

        # Update the numerators and the denominator
        numerator_A += weight * A
        numerator_B += weight * B
        numerator_C += weight * C
        denominator += weight

    # Compute the consensus matrices (now no need for epsilon)
    consensus_A = numerator_A / denominator
    consensus_B = numerator_B / denominator
    consensus_C = numerator_C / denominator

    return {'A': consensus_A, 'B': consensus_B, 'C': consensus_C}


In [101]:
"""def compute_consensus_point(particles, alpha):
    numerator_A = np.zeros_like(particles[0]['A'])
    numerator_B = np.zeros_like(particles[0]['B'])
    numerator_C = np.zeros_like(particles[0]['C'])
    denominator = 0.0

    for particle in particles:
        A, B, C = particle['A'], particle['B'], particle['C']

        # Compute the energy for the current particle (Frobenius norm error)
        energy = objective_function(particle)

        # Compute the weight for this particle
        weight = np.exp(-alpha * energy)

        # Update the numerators and the denominator
        numerator_A += weight * A
        numerator_B += weight * B
        numerator_C += weight * C
        denominator += weight

    # Add a small epsilon to avoid division by zero
    epsilon = 1e-10
    denominator = np.maximum(denominator, epsilon)

    # Compute the consensus matrices
    consensus_A = numerator_A / denominator
    consensus_B = numerator_B / denominator
    consensus_C = numerator_C / denominator

    return {'A': consensus_A, 'B': consensus_B, 'C': consensus_C}"""

"def compute_consensus_point(particles, alpha):\n    numerator_A = np.zeros_like(particles[0]['A'])\n    numerator_B = np.zeros_like(particles[0]['B'])\n    numerator_C = np.zeros_like(particles[0]['C'])\n    denominator = 0.0\n\n    for particle in particles:\n        A, B, C = particle['A'], particle['B'], particle['C']\n\n        # Compute the energy for the current particle (Frobenius norm error)\n        energy = objective_function(particle)\n\n        # Compute the weight for this particle\n        weight = np.exp(-alpha * energy)\n\n        # Update the numerators and the denominator\n        numerator_A += weight * A\n        numerator_B += weight * B\n        numerator_C += weight * C\n        denominator += weight\n\n    # Add a small epsilon to avoid division by zero\n    epsilon = 1e-10\n    denominator = np.maximum(denominator, epsilon)\n\n    # Compute the consensus matrices\n    consensus_A = numerator_A / denominator\n    consensus_B = numerator_B / denominator\n    con

In [102]:
def projection_operator_matrix(V):
    """Apply the projection operator P(V) = I - (V ⊗ V) / |V|^2 to each column of the matrix V."""
    V_norm_sq = np.sum(V**2, axis=0, keepdims=True)  # Compute |V|^2 for each column (1 x n_columns)
    outer_product = np.einsum('ik,jk->ijk', V, V)  # Compute V ⊗ V for each column (n_rows x n_rows x n_columns)
    I = np.eye(V.shape[0])  # Identity matrix of appropriate size (n_rows x n_rows)
    P = I[:, :, np.newaxis] - outer_product / V_norm_sq  # Apply the projection operator column-wise
    return P

In [103]:
def isotropic_update(particles, consensus_point, lambda_, sigma, dt):
    for particle in particles:
        """
        Perform isotropic updates on all particles.

        Parameters:
        - particles: list of particle dictionaries with keys 'A', 'B', 'C'.
        - consensus_point: dictionary with keys 'A', 'B', 'C' representing the consensus matrices.
        - lambda_: drift parameter.
        - sigma: diffusion parameter.
        - dt: time step.
        """

        # Extract the matrices A, B, C from the particle and the consensus
        A, B, C = particle['A'], particle['B'], particle['C']
        A_consensus, B_consensus, C_consensus = consensus_point['A'], consensus_point['B'], consensus_point['C']

        if objective_function(compute_consensus_point(particles, alpha)) < objective_function(particle):
            drift_A = (-lambda_) * (A - A_consensus) * dt
            drift_B = (-lambda_) * (B - B_consensus) * dt
            drift_C = (-lambda_) * (C - C_consensus) * dt
        else:
            drift_A = np.zeros(A.shape).tolist()
            drift_B = np.zeros(B.shape).tolist()
            drift_C = np.zeros(C.shape).tolist()

        # Isotropic noise term, applied column_wise
        noise_A = sigma * (A - A_consensus) * np.random.randn(*A.shape) * dt
        noise_B = sigma * (B - B_consensus) * np.random.randn(*B.shape) * dt
        noise_C = sigma * (C - C_consensus) * np.random.randn(*C.shape) * dt

        # Update particle's A, B, and C matrices
        particle['A'] += np.array(drift_A) + np.array(noise_A)
        particle['B'] += np.array(drift_B) + np.array(noise_B)
        particle['C'] += np.array(drift_C) + np.array(noise_C)


In [104]:
def anisotropic_update(particles, consensus_point, lambda_, sigma, dt):
    for particle in particles:
        # Extract the matrices A, B, C from the particle and the consensus
        A, B, C = particle['A'], particle['B'], particle['C']
        A_consensus, B_consensus, C_consensus = consensus_point['A'], consensus_point['B'], consensus_point['C']

        # Compute the elementwise differences
        diff_A = A - A_consensus
        diff_B = B - B_consensus
        diff_C = C - C_consensus

        # Compute the projection terms for each matrix
        P_A = projection_operator_matrix(A)
        P_B = projection_operator_matrix(B)
        P_C = projection_operator_matrix(C)

        # Generate Brownian motion term
        delta_B_A = np.random.randn(*A.shape) * np.sqrt(dt)
        delta_B_B = np.random.randn(*B.shape) * np.sqrt(dt)
        delta_B_C = np.random.randn(*C.shape) * np.sqrt(dt)

        # Corrected drift term: includes projection P(V_ti) applied column-wis
        drift_A = lambda_ * dt * np.einsum('ijk,jk->ik', P_A, A_consensus)
        drift_B = lambda_ * dt * np.einsum('ijk,jk->ik', P_B, B_consensus)
        drift_C = lambda_ * dt * np.einsum('ijk,jk->ik', P_C, C_consensus)

        # Anisotropic noise term, projected and applied column-wise
        D_A = np.diag(np.linalg.norm(diff_A, axis=0)**2)
        noise_A = sigma * np.matmul(diff_A, D_A) * delta_B_A

        D_B = np.diag(np.linalg.norm(diff_B, axis=0)**2)
        noise_B = sigma * np.matmul(diff_B, D_B) * delta_B_B

        D_C = np.diag(np.linalg.norm(diff_C, axis=0)**2)
        noise_C = sigma * np.matmul(diff_C, D_C) * delta_B_C

        # Additional correction term applied column-wise
        correction_A = -0.5 * dt * sigma**2 * (np.linalg.norm(diff_A, axis=0, keepdims=True)**2 * A)
        correction_B = -0.5 * dt * sigma**2 * (np.linalg.norm(diff_B, axis=0, keepdims=True)**2 * B)
        correction_C = -0.5 * dt * sigma**2 * (np.linalg.norm(diff_C, axis=0, keepdims=True)**2 * C)

        # Update particle's A, B, and C matrices
        particle['A'] += drift_A + noise_A + correction_A
        particle['B'] += drift_B + noise_B + correction_B
        particle['C'] += drift_C + noise_C + correction_C

In [105]:
# List to store reconstruction errors of the consensus point at each iteration
consensus_errors = []

# Main iteration loop
for iteration in range(n_iterations):
    # Step 1: Calculate the consensus point
    consensus_point = compute_consensus_point(particles, alpha=alpha)

    # Step 2: Compute the reconstruction error for the consensus point
    consensus_tensor = cp_to_tensor((np.ones(rank), [consensus_point['A'], consensus_point['B'], consensus_point['C']]))
    consensus_error = tl.norm(tensor - consensus_tensor)

    # Print the reconstruction error of the consensus point
    print(f"Iteration {iteration + 1}/{n_iterations}, Consensus Reconstruction Error: {consensus_error}")

    # Store the error for later plotting
    consensus_errors.append(consensus_error)

    # Step 3: Update the particles
    isotropic_update(particles, consensus_point, lambda_, sigma, dt)
    # or use anisotropic_update(particles, consensus_point, lambda_, sigma, dt) if desired

# Final output: You could also save or analyze the final consensus point
final_consensus_point = compute_consensus_point(particles, alpha=alpha)
final_consensus_tensor = cp_to_tensor((np.ones(rank), [final_consensus_point['A'], final_consensus_point['B'], final_consensus_point['C']]))
final_error = tl.norm(tensor - final_consensus_tensor)

print("Final Consensus Reconstruction Error: ", final_error)

# Plotting the consensus reconstruction errors over iterations
plt.plot(consensus_errors, label='Consensus Reconstruction Error')
plt.xlabel('Iteration')
plt.ylabel('Reconstruction Error')
plt.title('Reconstruction Error of Consensus Point Over Iterations')
plt.legend()
plt.grid(True)
plt.show()

Iteration 1/1000, Consensus Reconstruction Error: 52.45222447596814


KeyboardInterrupt: 